## Part 1 Prompt

--- Day 2: 1202 Program Alarm ---
On the way to your gravity assist around the Moon, your ship computer beeps angrily about a "1202 program alarm". On the radio, an Elf is already explaining how to handle the situation: "Don't worry, that's perfectly norma--" The ship computer bursts into flames.

You notify the Elves that the computer's magic smoke seems to have escaped. "That computer ran Intcode programs like the gravity assist program it was working on; surely there are enough spare parts up there to build a new Intcode computer!"

An Intcode program is a list of integers separated by commas (like 1,0,0,3,99). To run one, start by looking at the first integer (called position 0). Here, you will find an opcode - either 1, 2, or 99. The opcode indicates what to do; for example, 99 means that the program is finished and should immediately halt. Encountering an unknown opcode means something went wrong.

Opcode 1 adds together numbers read from two positions and stores the result in a third position. The three integers immediately after the opcode tell you these three positions - the first two indicate the positions from which you should read the input values, and the third indicates the position at which the output should be stored.

For example, if your Intcode computer encounters 1,10,20,30, it should read the values at positions 10 and 20, add those values, and then overwrite the value at position 30 with their sum.

Opcode 2 works exactly like opcode 1, except it multiplies the two inputs instead of adding them. Again, the three integers after the opcode indicate where the inputs and outputs are, not their values.

Once you're done processing an opcode, move to the next one by stepping forward 4 positions.

For example, suppose you have the following program:

1,9,10,3,2,3,11,0,99,30,40,50
For the purposes of illustration, here is the same program split into multiple lines:

1,9,10,3,
2,3,11,0,
99,
30,40,50
The first four integers, 1,9,10,3, are at positions 0, 1, 2, and 3. Together, they represent the first opcode (1, addition), the positions of the two inputs (9 and 10), and the position of the output (3). To handle this opcode, you first need to get the values at the input positions: position 9 contains 30, and position 10 contains 40. Add these numbers together to get 70. Then, store this value at the output position; here, the output position (3) is at position 3, so it overwrites itself. Afterward, the program looks like this:

1,9,10,70,
2,3,11,0,
99,
30,40,50
Step forward 4 positions to reach the next opcode, 2. This opcode works just like the previous, but it multiplies instead of adding. The inputs are at positions 3 and 11; these positions contain 70 and 50 respectively. Multiplying these produces 3500; this is stored at position 0:

3500,9,10,70,
2,3,11,0,
99,
30,40,50
Stepping forward 4 more positions arrives at opcode 99, halting the program.

Here are the initial and final states of a few more small programs:

1,0,0,0,99 becomes 2,0,0,0,99 (1 + 1 = 2).
2,3,0,3,99 becomes 2,3,0,6,99 (3 * 2 = 6).
2,4,4,5,99,0 becomes 2,4,4,5,99,9801 (99 * 99 = 9801).
1,1,1,4,99,5,6,0,99 becomes 30,1,1,4,2,5,6,0,99.
Once you have a working computer, the first step is to restore the gravity assist program (your puzzle input) to the "1202 program alarm" state it had just before the last computer caught fire. To do this, before running the program, replace position 1 with the value 12 and replace position 2 with the value 2. What value is left at position 0 after the program halts?

In [1]:
input_line_1 = "1,9,10,3"

In [2]:
input_line_1_codes = [int(x) for x in input_line_1.split(",")]
input_line_1_codes

[1, 9, 10, 3]

In [3]:
if input_line_1_codes[0] == 1:
    input_int_1 = input_line_1_codes[1]
    input_int_2 = input_line_1_codes[2]
    output_int = input_int_1 + input_int_2
    output_codes = input_line_1_codes[:-1]
    output_codes.append(output_int)
    print(output_codes)

[1, 9, 10, 19]


In [4]:
output_line_1 = ",".join([str(x) for x in output_codes])
output_line_1

'1,9,10,19'

Oh, dangit, it doesn't always replace the next one, the number itself tells you where to replace.  That's much more annoying

In [5]:
program = "1,9,10,3,2,3,11,0,99,30,40,50"

In [6]:
program_codes = [int(x) for x in program.split(",")]
program_codes

[1, 9, 10, 3, 2, 3, 11, 0, 99, 30, 40, 50]

In [7]:
current_position = 0
while True:
    opcode = program_codes[current_position]
    if opcode == 99:
        break
    else:
        input_1_index = program_codes[current_position+1]
        input_2_index = program_codes[current_position+2]
        
        input_1 = program_codes[input_1_index]
        input_2 = program_codes[input_2_index]
        
        destination_index = program_codes[current_position+3]
        if opcode == 1:
            output = input_1 + input_2
        elif opcode == 2:
            output = input_1 * input_2
        else:
            print("Something went wrong, got opcode", opcode)
            break
        print("Position", destination_index, "is now set to", output)
        program_codes[destination_index] = output
        current_position = current_position + 4

Position 3 is now set to 70
Position 0 is now set to 3500


In [8]:
program_codes

[3500, 9, 10, 70, 2, 3, 11, 0, 99, 30, 40, 50]

In [9]:
def run_program(program):
    # split the string into a bunch of ints
    program_codes = [int(x) for x in program.split(",")]
    
    # loop through codes in 4-code "programs"
    current_position = 0
    while True:
        opcode = program_codes[current_position]
        if opcode == 99:
            break
        else:
            # for inputs, first we have to get their index, then we have
            # to get the values at that index
            input_1_index = program_codes[current_position+1]
            input_2_index = program_codes[current_position+2]

            input_1 = program_codes[input_1_index]
            input_2 = program_codes[input_2_index]

            # for output, we only need the index, since we are overwriting
            # whatever is there
            destination_index = program_codes[current_position+3]
            
            # compute the output
            if opcode == 1:
                output = input_1 + input_2
            elif opcode == 2:
                output = input_1 * input_2
            else:
                print("Something went wrong, got opcode", opcode)
                break
            
            # set the destination index to the output
            print("Position", destination_index, "is now set to", output)
            program_codes[destination_index] = output
            
            # move to the next "program"
            current_position = current_position + 4
            
    print("Final program codes:", ",".join([str(x) for x in program_codes]))
    

In [10]:
run_program("1,9,10,3,2,3,11,0,99,30,40,50")

Position 3 is now set to 70
Position 0 is now set to 3500
Final program codes: 3500,9,10,70,2,3,11,0,99,30,40,50


In [11]:
run_program("1,0,0,0,99")

Position 0 is now set to 2
Final program codes: 2,0,0,0,99


In [12]:
run_program("2,3,0,3,99")

Position 3 is now set to 6
Final program codes: 2,3,0,6,99


In [13]:
run_program("2,4,4,5,99,0")

Position 5 is now set to 9801
Final program codes: 2,4,4,5,99,9801


In [14]:
run_program("1,1,1,4,99,5,6,0,99")

Position 4 is now set to 2
Position 0 is now set to 30
Final program codes: 30,1,1,4,2,5,6,0,99


In [15]:
part1_input_file = open("input.txt", "r")
part1_input = part1_input_file.readline()
part1_input

'1,0,0,3,1,1,2,3,1,3,4,3,1,5,0,3,2,13,1,19,1,9,19,23,2,23,13,27,1,27,9,31,2,31,6,35,1,5,35,39,1,10,39,43,2,43,6,47,1,10,47,51,2,6,51,55,1,5,55,59,1,59,9,63,1,13,63,67,2,6,67,71,1,5,71,75,2,6,75,79,2,79,6,83,1,13,83,87,1,9,87,91,1,9,91,95,1,5,95,99,1,5,99,103,2,13,103,107,1,6,107,111,1,9,111,115,2,6,115,119,1,13,119,123,1,123,6,127,1,127,5,131,2,10,131,135,2,135,10,139,1,13,139,143,1,10,143,147,1,2,147,151,1,6,151,0,99,2,14,0,0\n'

before running the program, replace position 1 with the value 12 and replace position 2 with the value 2.

In [16]:
part1_input = part1_input.strip()
part1_input_codes = [int(x) for x in part1_input.split(",")]
part1_input_codes[1] = 12
part1_input_codes[2] = 2
part1_input = ",".join([str(x) for x in part1_input_codes])
part1_input

'1,12,2,3,1,1,2,3,1,3,4,3,1,5,0,3,2,13,1,19,1,9,19,23,2,23,13,27,1,27,9,31,2,31,6,35,1,5,35,39,1,10,39,43,2,43,6,47,1,10,47,51,2,6,51,55,1,5,55,59,1,59,9,63,1,13,63,67,2,6,67,71,1,5,71,75,2,6,75,79,2,79,6,83,1,13,83,87,1,9,87,91,1,9,91,95,1,5,95,99,1,5,99,103,2,13,103,107,1,6,107,111,1,9,111,115,2,6,115,119,1,13,119,123,1,123,6,127,1,127,5,131,2,10,131,135,2,135,10,139,1,13,139,143,1,10,143,147,1,2,147,151,1,6,151,0,99,2,14,0,0'

In [17]:
run_program(part1_input)

Position 3 is now set to 3
Position 3 is now set to 14
Position 3 is now set to 15
Position 3 is now set to 2
Position 19 is now set to 60
Position 23 is now set to 63
Position 27 is now set to 315
Position 31 is now set to 318
Position 35 is now set to 636
Position 39 is now set to 637
Position 43 is now set to 641
Position 47 is now set to 1282
Position 51 is now set to 1286
Position 55 is now set to 2572
Position 59 is now set to 2573
Position 63 is now set to 2576
Position 67 is now set to 2581
Position 71 is now set to 5162
Position 75 is now set to 5163
Position 79 is now set to 10326
Position 83 is now set to 20652
Position 87 is now set to 20657
Position 91 is now set to 20660
Position 95 is now set to 20663
Position 99 is now set to 20664
Position 103 is now set to 20665
Position 107 is now set to 103325
Position 111 is now set to 103327
Position 115 is now set to 103330
Position 119 is now set to 206660
Position 123 is now set to 206665
Position 127 is now set to 206667
Posit

## Part 2 Prompt

--- Part Two ---
"Good, the new computer seems to be working correctly! Keep it nearby during this mission - you'll probably use it again. Real Intcode computers support many more features than your new one, but we'll let you know what they are as you need them."

"However, your current priority should be to complete your gravity assist around the Moon. For this mission to succeed, we should settle on some terminology for the parts you've already built."

Intcode programs are given as a list of integers; these values are used as the initial state for the computer's memory. When you run an Intcode program, make sure to start by initializing memory to the program's values. A position in memory is called an address (for example, the first value in memory is at "address 0").

Opcodes (like 1, 2, or 99) mark the beginning of an instruction. The values used immediately after an opcode, if any, are called the instruction's parameters. For example, in the instruction 1,2,3,4, 1 is the opcode; 2, 3, and 4 are the parameters. The instruction 99 contains only an opcode and has no parameters.

The address of the current instruction is called the instruction pointer; it starts at 0. After an instruction finishes, the instruction pointer increases by the number of values in the instruction; until you add more instructions to the computer, this is always 4 (1 opcode + 3 parameters) for the add and multiply instructions. (The halt instruction would increase the instruction pointer by 1, but it halts the program instead.)

"With terminology out of the way, we're ready to proceed. To complete the gravity assist, you need to determine what pair of inputs produces the output 19690720."

The inputs should still be provided to the program by replacing the values at addresses 1 and 2, just like before. In this program, the value placed in address 1 is called the noun, and the value placed in address 2 is called the verb. Each of the two input values will be between 0 and 99, inclusive.

Once the program has halted, its output is available at address 0, also just like before. Each time you try a pair of inputs, make sure you first reset the computer's memory to the values in the program (your puzzle input) - in other words, don't reuse memory from a previous attempt.

Find the input noun and verb that cause the program to produce the output 19690720. What is 100 * noun + verb? (For example, if noun=12 and verb=2, the answer would be 1202.)

## My Thoughts

It seems like I can (and maybe am supposed to) just brute-force this?  Like, try each permutation of (0-99) and (0-99), making sure that I quit if there is an invalid code

I'm going to do this very inefficiently but very cautiously, by making a deep copy every time

In [18]:
target = 19690720

In [19]:
part1_input

'1,12,2,3,1,1,2,3,1,3,4,3,1,5,0,3,2,13,1,19,1,9,19,23,2,23,13,27,1,27,9,31,2,31,6,35,1,5,35,39,1,10,39,43,2,43,6,47,1,10,47,51,2,6,51,55,1,5,55,59,1,59,9,63,1,13,63,67,2,6,67,71,1,5,71,75,2,6,75,79,2,79,6,83,1,13,83,87,1,9,87,91,1,9,91,95,1,5,95,99,1,5,99,103,2,13,103,107,1,6,107,111,1,9,111,115,2,6,115,119,1,13,119,123,1,123,6,127,1,127,5,131,2,10,131,135,2,135,10,139,1,13,139,143,1,10,143,147,1,2,147,151,1,6,151,0,99,2,14,0,0'

In [20]:
def run_list_program(program_codes):
    # loop through codes in 4-code "programs"
    current_position = 0
    while True:
        opcode = program_codes[current_position]
        if opcode == 99:
            break
        else:
            # for inputs, first we have to get their index, then we have
            # to get the values at that index
            input_1_index = program_codes[current_position+1]
            input_2_index = program_codes[current_position+2]

            input_1 = program_codes[input_1_index]
            input_2 = program_codes[input_2_index]

            # for output, we only need the index, since we are overwriting
            # whatever is there
            destination_index = program_codes[current_position+3]
            
            # compute the output
            if opcode == 1:
                output = input_1 + input_2
            elif opcode == 2:
                output = input_1 * input_2
            else:
                print("Something went wrong, got opcode", opcode)
                break
            
            # set the destination index to the output
#             print("Position", destination_index, "is now set to", output)
            program_codes[destination_index] = output
            
            # move to the next "program"
            current_position = current_position + 4

In [21]:
all_number_permutations = [(noun, verb) for noun in range(100) for verb in range(100)]

part1_input_list = [int(x) for x in part1_input.split(",")]

for noun, verb in all_number_permutations:
    current_input_attempt = part1_input_list[:]
    current_input_attempt[1] = noun
    current_input_attempt[2] = verb
    
    run_list_program(current_input_attempt)
    if current_input_attempt[0] == target:
        print("Found noun", noun, "and verb", verb)
        break

Found noun 76 and verb 21


100 * noun + verb

In [22]:
100 * 76 + 21

7621